In [17]:
from bs4 import BeautifulSoup
import xlwt
import openpyxl


In [18]:
def loadSeason(season):
    ecj_data = open(season,'r').read()
    soup = BeautifulSoup(ecj_data)
    
    list_of_teams_temp = soup.findAll("span", { "class" : "padl" })
    results_temp = soup.findAll("td", { "class" : "cell_sa score  bold" })
    
    list_of_teams = []
    for team in list_of_teams_temp:
        x = team.get_text();
        list_of_teams.append(x.encode("utf-8"))
    
    results = []
    for result in results_temp:
        if result.get_text()=="-":
            home_team_score = 0
            away_team_score = 0
        else:
            home_team_score,away_team_score = result.get_text().split(':')
        
        results.append(int(home_team_score))
        results.append(int(away_team_score))

    list_of_teams = list_of_teams[::-1] # obrcemo redosled jer se u .html fajlu utakmice zapisane od 34-og kola ka 1-om
    results = results[::-1]
    
    return list_of_teams,results

In [19]:
def writeSeasonInExcel(file_name,list_of_teams,results):
    book = xlwt.Workbook()
    sh = book.add_sheet("Sheet 1")

    for i in range(0,len(list_of_teams),2):
        if ("Germany" in list_of_teams[i+1]) or ("Bundesliga Stars" in list_of_teams[i]): # u .html fajlu se nalazi i ALL-STAR utakmica
            continue
        sh.write(i/2,0,list_of_teams[i+1])
        sh.write(i/2,1,list_of_teams[i])
        sh.write(i/2,2,results[i+1])
        sh.write(i/2,3,results[i])
    
    book.save(file_name)

In [20]:
NUMBER_OF_SEASONS = 7
start_season_begin_year = 2014
start_season_end_year = 2015

for i in range(0,NUMBER_OF_SEASONS):
    seasonName = "DataSet/season" + str(start_season_begin_year-i) + "-" + str(start_season_end_year-i) 
    list_of_teams,results = loadSeason(seasonName + ".html")
    writeSeasonInExcel(seasonName + ".xls",list_of_teams,results)


In [21]:
def startProcessingData():
        NUMBER_OF_SEASONS = 7
        NUMBER_OF_PREVIOUS_GAMES = 6
        MR_MAX = 100 #Match rating maximum
        
        workbook = openpyxl.load_workbook("DataSet/list_of_all_teams.xlsx", use_iterators=True)
        sheet = workbook.worksheets[0]
        number_of_all_teams = sheet.get_highest_row() - 1 
        
        current_standing = {}
        previous_standing1 = {} #standing of one season before current season
        previous_standing2 = {} #standing of two season before current season
        previous_standing3 = {} #standing of three season before current season
        goal_difference = {} 
        
        for row in sheet.iter_rows():
            for k in row:
                current_standing[k.internal_value.encode("utf-8")] = 0 
                previous_standing1[k.internal_value.encode("utf-8")] = 0
                previous_standing2[k.internal_value.encode("utf-8")] = 0
                previous_standing3[k.internal_value.encode("utf-8")] = 0
                goal_difference[k.internal_value.encode("utf-8")] = [0] * NUMBER_OF_PREVIOUS_GAMES
        
       
        return goal_difference

In [22]:
goal_difference = startProcessingData()
